# HW2 수식 최적화를 작은걸음 의미구조로 표현 (PL 2025 Fall @ HNU CE)

- 이름: 이재석
- 학번: 20210492

과제를 하기 전에 위에다 학번과 이름을 꼭 작성하기

In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables #-}

import Data.List -- nub, sort, union, foldl, foldr, foldl1, foldr1, (\\), ...
-- 더 자세한 내용은 https://hackage.haskell.org/package/base-4.17.1.0/docs/Data-List.html

In [3]:
:!ghc -- --version

The Glorious Glasgow Haskell Compilation System, version 9.4.5

In [4]:
data Expr = I Int         -- n 정수
          | V String      -- x 변수
          | Add Expr Expr -- e + e 덧셈식
          | Mul Expr Expr -- e × e 곱셈식
    deriving (Eq,Ord,Show)

최적화 핵심 규칙을 수식과 한국어로

- $n_1+n_2$를 $n$으로 최적화 (단, $n\equiv n_1+n_2$ 정수 덧셈 결과)
- $n_1\times n_2$를 $n$으로 최적화 (단, $n\equiv n_1\times n_2$ 정수 곱셈 결과)
- $e_1+0$를 $e_1$으로, $0+e_2$를 $e_2$로 최적화
- $e_1\times 1$를 $e_1$으로, $1\times e_2$를 $e_2$로 최적화
- $e_1\times 0$를 $0$으로, $0\times e_2$를 $0$으로 최적화


작은걸음 동작과정 의미구조(small-step operational semantics)로 최적화 핵심 규칙을 표현

$\displaystyle\frac{}{n_1 + n_2 \longmapsto n}~(\text{단,}~n\equiv n_1+n_2)$

$\displaystyle\frac{}{n_1 \times n_2 \longmapsto n}~(\text{단,}~n\equiv n_1\times n_2)$

$\displaystyle\frac{}{e_1 + 0 \longmapsto e_1} \qquad \frac{}{0 + e_2 \longmapsto e_2}$

$\displaystyle\frac{}{e_1 \times 0 \longmapsto 0} \qquad \frac{}{0 \times e_2 \longmapsto 0}$

$\displaystyle\frac{}{e_1 \times 1 \longmapsto e_1} \qquad \frac{}{1 \times e_2 \longmapsto e_2}$

---
### 1. 핵심적 규칙 외에 필요한 맥락규칙을 추론규칙의 형태로 작성해 보기 (2점)
특정 맥락을 우선시하지 않는 가장 기본적인 형태의 비결정적인 의미구조로 작성.

수식 기능을 활용해서 작성해도 좋고 수식 작성이 어렵게 느껴지는 사람들은 아래와 같이 아스키 그림 형태로 작성해도 됨.


```
조건1   조건2
-------------
 e |--> e1
```

**주의**: 이 문제를 정답으로 풀어내지 못하면 전체 과제 점수가 0점이다. 여기서 작성한 의미구조 규칙대로 프로그램을 짜는 과제인데, 가장 기본이 되는 내용이 없으면 무의미하기 때문. (무슨 스펙을 기준으로 코딩? 스펙이 없는데? ... 라는 상황) 따라서 이 문제가 점수 배점이 낮으니까 대충 넘어가고 나머지 프로그래밍이나 해야지라고 생각하면 전체 과제 점수가 0점이 되기에 딱 좋은 접근 방식이다. 이 문제만은 절대로 틀리지 않고 검토에 검토, 재검토, 재재검토, 재재재검토, 재재재재재검토를 거쳐 반드시 정확하게 작성하고 다른 문제를 풀겠다는 자세로 접근하는 것을 매우 강력하게 권장함.

여기에 맥락 규칙들 작성

e1 |--> e1'
----------------- (왼쪽더하기)
e1 + e2 |--> e1' + e2

e2 |--> e2'
----------------- (오른쪽더하기)
e1 + e2 |--> e1 + e2'

e1 |--> e1'
----------------- (왼쪽 곱하기)
e1 * e2 |--> e1' * e2

e2 |--> e2'
-----------------(오른쪽 곱하기)
e1 * e2 |--> e1 * e2'

여기에 맥락 규칙들 작성

---

In [216]:
-- 식(Expr)을 적절히 알아보기 좋게 출력할 만한 문자열로 변환하는 함수
expr2str :: Expr -> String
expr2str (I n)
        | n >= 0     = show n
        | otherwise  = "(" ++ show n ++ ")" --음수 결과확인을 위해 일부 수정함
expr2str (V x)       = x
expr2str (Add e1 e2) = paren (expr2str e1 ++ "+" ++ expr2str e2) 
expr2str (Mul e1 e2) = paren (expr2str e1 ++ "*" ++ expr2str e2)

paren s = "("++s++")"

---
### 2. 위의 최적화 규칙들에 따라 1단계 진행시키는 함수 작성 (6점)
앞서 제시된 핵심규칙과 위의 1번 문제에서 작성한 맥락규칙에 따라 작은걸음 의미구조를 한 단계 진행시키는 `step :: Expr -> [Expr]` 함수의 형태로 작성.

필요하다면 적절한 보조함수 혹은 지역함수 등을 정의하여 그것들을 활용하여 작성해도 좋다. 테스트 결과를 알아보기 좋게 `expr2str` 함수 활용을 권장.

작성하고 적절히 테스트 실행. 최소 4개 이상의 적절히 다양한 식들로 테스트해 볼 것.

In [198]:
-- step :: Expr -> [Expr]
step :: Expr -> [Expr]
step (I n) = []
step (V x) = []
step (Add e1 e2) = 
    case (e1, e2) of
        (I n1, I n2) -> [I (n1 + n2)]
        (_, I 0)     -> [e1]
        (I 0, _)     -> [e2]
        _            -> leftSteps ++ rightSteps
  where
    leftSteps  = map (\e1' -> Add e1' e2) $ step e1
    rightSteps = map (\e2' -> Add e1 e2') $ step e2
step (Mul e1 e2) = 
    case (e1, e2) of
        (I n1, I n2) -> [I (n1 * n2)]
        (_, I 0)     -> [I 0]
        (I 0, _)     -> [I 0]
        (_, I 1)     -> [e1]
        (I 1, _)     -> [e2]
        _            -> leftSteps ++ rightSteps
  where
    leftSteps  = map (\e1' -> Mul e1' e2) $ step e1
    rightSteps = map (\e2' -> Mul e1 e2') $ step e2
step _ = []

In [199]:
-- 적절히 테스트 왼쪽 더하기
test2_1 = Add (Add (I 2) (I 4)) (V "x")
stepTest e = map expr2str (step e)
stepTest test2_1

["(6+x)"]

In [200]:
-- 적절히 테스트 오른쪽 더하기
test2_2 = Add (I 8) (Add (I 5) (I 4))
stepTest test2_2

["(8+9)"]

In [201]:
-- 적절히 테스트 오른쪽 곱하기
test2_3 = Mul (I 4) (Mul (I 5) (I 6))
stepTest test2_3

["(4*30)"]

In [219]:
-- 적절히 테스트 왼쪽 곱하기
test2_4 = Mul (Mul (I 4) (I 0)) (I 10)
stepTest test2_4

["(0*10)"]

---
### 3. step을 활용해 더 이상 최적화할 수 없는 단계까지 최대한 진행시키는 함수 작성 (4점)
`opt` 함수의 계산 결과는 최적화 작은걸음이 몇 단계나 진행되었는지 그 정수값(`Int`)과 최적화된 식(`Expr`)의 순서쌍이다.

최적화 걸음을 적용할 수 없는 경우에는 `opt e0 == (0,e0)`가 되어야 한다.

함수를 작성하고 적어도 4개 이상의 적절히 다양한 식들로 테스트해 보라. 테스트 결과를 알아보기 좋게 `expr2str` 함수 활용을 권장.

**힌트**: 교재의 `hat` 함수, 하스켈 표준라이브러리 `takeWhile`, `length`, `last` 함수들

In [243]:
 -- 함수를 작성하라. 필요하다면 여러 개의 등식으로 나누어 작성하라.
opt :: Expr -> (Int, Expr)
opt e0 = go 0 e0
  where
    go n e = case step e of
        []   -> (n, e)
        (e':_) -> go (n+1) e'

In [244]:
-- 적절히 테스트
test3_1 = Add (Add (I 2) (I (-4))) (I 1)
optTest e = let (n, e') = opt e in (n, expr2str e')
optTest test3_1

(2,"(-1)")

In [245]:
-- 적절히 테스트
test3_2 = Add (Mul (I 2) (I 3)) (Mul (I 4) (I 0))
optTest test3_2

(3,"6")

In [246]:
-- 적절히 테스트
test3_3 = Add (Add (I 1) (V "x")) (Add (V "y") (I 2))
optTest e = let (n, e') = opt e in (n, expr2str e')
optTest test3_3

(0,"((1+x)+(y+2))")

In [247]:
-- 적절히 테스트
test3_4 = Add (Mul (V "x") (I 0)) (V "y")
optTest test3_4

(2,"y")

------
### 4. 덧셈의 교환법칙과 결합법칙을 적절히 적용해 최적화를 가능한 한 더 많이 진행시킬 수 있는 형태로 변형된 수식을 계산하는 함수 작성 (8점)

우리가 상식적으로 알고 있는 덧셈의 교환법칙과 덧셈의 결합법칙을 추론규칙의 형태로 굳이 나타내 보자면 다음과 같이 표현할 수 있다.

$\displaystyle
 (\textsf{symm})\,\frac{}{e_1 + e_2 \sim e_2 + e_1}
 \qquad
 (\textsf{comm})\,\frac{}{(e_1+e_2)+e_3 \sim e_1+(e_2+e_3)}$

곱셈의 교환법칙과 결합법칙은 이번 과제에서는 활용할 필요가 없으므로 추론규칙의 형태고 작성하는 것을 생략함.

물론 교환법칙과 결합법칙이 복잡한 수식의 안쪽 부분에도 적용할 수 있어야 하므로 $\sim$에 대한 맥략규칙도 물론 필요할 것이지만, 여기서는 그렇게까지 자세히 교환법칙과 결합법칙에 대한 추론규칙 하나하나를 따져볼 것은 아니므로 이것도 역시 작성을 생략함.

곱셉식과 덧셈식은 교환법칙과 결합법칙이 성립한다는 성질을 활용하면 위의 규칙대로만으로 최적화하는 것보다 더 많은 단계의 최적화가 가능하다.

예컨대, $(1 + x) + ((y +  \textrm{-}3) + (z + 2))$는 앞서 추론규칙으로 작성한 최적화의 의미구조대로는 더 이상 최적화를 진행할 수 없다.

하지만, 덧셈에 대한 교환법칙과 결합법칙을 적절히 잘 활용하면 다음과 같이 덧셈식을 재구성할 수 있으므로 \
$(1 + x) + ((y +  \textrm{-}3) + (z + 2)) \sim ((1 + \textrm{-}3) + 2) + ((x+y)+z)$ \
더 많은 단계의 최적화를 아래와 같이 진행시킬 수 있다.\
$((1 + \textrm{-}3) + 2) + ((x+y)+z) \longmapsto (\textrm{-}2 + 2) + ((x+y)+z) \longmapsto 0 + ((x+y)+z) \longmapsto (x+y)+z$


그래서 이런 식으로 $(1 + x) + ((y +  \textrm{-}3) + (z + 2))$와 같은 식이 주어지면,
교환법칙과 결합법칙이 적절히 적용된 $((1 + \textrm{-}3) + 2) + ((x+y)+z)$와 같이 최적화를 가능한 많이 진행시킬 수 있는 변형된 형태의 식을 계산하는
`refactor` 함수를 작성하라. 필요하다면 보조 함수를 정의하거나 라이브러리 함수를 활용하라.

당연한 이야기지만, 이런 식으로 변형의 대상이 되는 덧셈식이 항상 식의 최상위 레벨에 있지 않을 수도 있다. 예를 들면 괄호로 묶여 곱셈식 안에 깊숙히 들어 있는 덧셈식도 이런 변형의 대상이 될 수 있다는 말이다.

테스트는 다음과 같은 방식으로 진행하라.
1. 테스트를 위해 주어진 식 `e0`를 `opt`로 최적화한 결과 `(n1,e1)`를 계산. (즉, `(n1,e1) = opt e0`)
1. `e1`을 `refactor`한 결과 `e2`를 계산. (즉, `e2 = opt e1`)
1. `n1` 그리고 `expr2str` 함수를 활용해 `e0`, `e1`, `e2`를 알아보기 좋은 방식으로 각각 한 줄씩 줄바꾸어 가며 출력하여 확인

최소 4개 이상의 적절히 다양한 식에 대해 테스트해 보라. 테스트들 중에는 `e0`, `e1`, `e2`가 모두 다른 경우가 반드시 포함되어 있어야 한다.

In [208]:
refactor :: Expr -> Expr
refactor (I n) = I n
refactor (V x) = V x
refactor (Add e1 e2) = 
    let e1' = refactor e1
        e2' = refactor e2
        terms = flattenAdd e1' ++ flattenAdd e2'
        (consts, vars) = partition isInt terms
        sumConst = sum [n | I n <- consts]
        newTerms = if sumConst /= 0 then I sumConst : vars else vars
    in rebuildAdd newTerms
refactor (Mul e1 e2) = 
    let e1' = refactor e1
        e2' = refactor e2
        (consts, vars) = partition isInt (flattenMul e1' ++ flattenMul e2')
        productConst = product [n | I n <- consts]
    in if productConst == 0 then I 0 else rebuildMul (I productConst : vars)

flattenAdd :: Expr -> [Expr]
flattenAdd (Add e1 e2) = flattenAdd e1 ++ flattenAdd e2
flattenAdd e = [e]

flattenMul :: Expr -> [Expr]
flattenMul (Mul e1 e2) = flattenMul e1 ++ flattenMul e2
flattenMul e = [e]

-- 정수 여부 확인
isInt :: Expr -> Bool
isInt (I _) = True
isInt _ = False

rebuildAdd :: [Expr] -> Expr
rebuildAdd [] = I 0
rebuildAdd [x] = x
rebuildAdd (x:xs) = Add x (rebuildAdd xs)

rebuildMul :: [Expr] -> Expr
rebuildMul [] = I 1
rebuildMul [x] = x
rebuildMul (x:xs) = Mul x (rebuildMul xs)

In [209]:
refactorTest e = expr2str (refactor e)

In [214]:
-- 적절히 테스트1 
test4_1_1 = Add (Add (I 1) (V "x")) (Add (Add (I 5) (I (-3))) (Add (V "z") (I 2))) -- e0 = (1+x)+((5+(-3)+(z+2))
optTest test4_1_1 -- opt로 최대로 최적화한 결과 e1 = (1+x)+(2+(z+2))

test4_1_2 = Add (Add (I 1) (V "x")) (Add (I 2) (Add (V "z") (I 2)))--e1을 refactor한 결과 e2 = 5+(x+z)
refactorTest test4_1_2

(1,"((1+x)+(2+(z+2)))")

"(5+(x+z))"

In [239]:
-- 적절히 테스트2
test4_2_1 = Add (Add (I 0) (I 7)) (Add (I (-2)) (V "x")) -- e0 = (0+7)+((-2)+x)
optTest test4_2_1 -- opt로 최대로 최적화한 결과 e1 = 7+((-2)+x)

test4_2_2 = Add (I 7) (Add (I (-2)) (V "x")) --e1을 refactor한 결과 e2 = 5+x
refactorTest test4_2_2

(1,"(7+((-2)+x))")

"(5+x)"

In [248]:
-- 적절히 테스트3
test4_3_1 = Add (Add (Add (V "x") (I 0)) (Add (I (-5)) (V "y"))) (Add (Add (I 3) (V "z")) (Add (I 0) (I 4))) -- e0 = ((x+0)+(-5+y))+(3+z)+(0+4)
optTest test4_3_1  -- opt로 최대로 최적화한 결과 e1 = (x+((-5)+y))+((3+z)+4)

test4_3_2 = Add (Add (V "x") (Add (I (-5)) (V "y"))) (Add (Add (I 3) (V "z")) (I 4)) 

refactorTest test4_3_2 --e1을 refactor한 결과 e2 = 2+(x+(y+z))

(2,"((x+((-5)+y))+((3+z)+4))")

"(2+(x+(y+z)))"

In [249]:
-- 적절히 테스트4
test4_4_1 = Add ( Add (Add (V "x") (I 0) ) ( Add (I (-5)) (V "y") )) ( Add ( Add (I 5) (I 4) ) ( Add (V "z") ( Add (I 0) (I 2) ) )) 
--e0 = ((x+0)+(−5+y))+((5+4)+(z+(0+2)))
optTest test4_4_1 --opt로 최대로 최적화한 결과 e1 = (x+((-5)+y))+(9+(z+2))

test4_4_2 =  Add (Add (V "x") (Add (I (-5)) (V "y") )) (Add (I 9) (Add (V "z") (I 2)))
refactorTest test4_4_2 --e1을 refactor한 결과 e2 = 6+(x+(y+z))

(3,"((x+((-5)+y))+(9+(z+2)))")

"(6+(x+(y+z)))"